In [1]:
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import requests
from requests import Session, request, get
from urllib.request import urlopen
import os
import pandas as pd
import numpy as np
from functools import reduce ## 추가 ##
import sqlite3

In [159]:
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26'}


In [160]:
con = sqlite3.connect("./musinsa_1101.db")
sql = "SELECT goods_id FROM 리뷰개수테이블 WHERE stylereviewCount >= 50 AND photoreviewCount >= 50;"
cur = con.cursor()
cur.execute(sql)
rows = cur.fetchall()
con.close()

In [161]:
goodcode_list = list()
for i in rows:
    goodcode_list.append(i[0])

In [162]:
len(goodcode_list)

5191

In [6]:
con = sqlite3.connect("./musinsa_1101.db")
sql = "SELECT * FROM 리뷰개수테이블 WHERE stylereviewCount >= 50 AND photoreviewCount >= 50;"
cur = con.cursor()
cur.execute(sql)
tuples = cur.fetchall()
con.close()

In [7]:
tuples

[(0, '947067', 17553, 2025, 3149),
 (1, '1208839', 17801, 2373, 3460),
 (2, '897632', 13611, 1441, 2356),
 (3, '1557508', 13548, 2124, 2974),
 (4, '956939', 11631, 1881, 2424),
 (5, '2038497', 11679, 1831, 2190),
 (6, '836499', 9171, 935, 1563),
 (7, '1582356', 11115, 1809, 2345),
 (8, '836495', 8724, 810, 1437),
 (9, '643173', 8467, 1230, 1544),
 (10, '1160367', 8520, 1171, 1989),
 (11, '836498', 7437, 901, 1382),
 (12, '1336836', 7833, 946, 1456),
 (13, '644037', 6747, 658, 1191),
 (14, '620634', 5903, 624, 1088),
 (15, '454425', 5826, 568, 939),
 (16, '2085371', 6903, 1072, 1238),
 (17, '1679943', 6668, 1135, 1364),
 (18, '947068', 5774, 712, 1154),
 (19, '1755609', 5544, 834, 1167),
 (20, '1126435', 4899, 625, 1018),
 (21, '1563623', 5065, 680, 1070),
 (22, '1557509', 4906, 729, 1048),
 (23, '1129701', 4216, 395, 835),
 (24, '1817394', 4752, 669, 828),
 (25, '1642887', 4483, 659, 881),
 (26, '1177545', 3819, 566, 790),
 (27, '1549281', 4335, 716, 1202),
 (28, '1759350', 4302, 730, 

In [61]:
tuples[5192]

IndexError: list index out of range

In [163]:
### 비율에 맞게 가중치, ###
### DB 에 쓸 코드임 ### 

url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list"
params = {
    'page' : None,
    'goodsNo' : None,
    'sort' : "comment_cnt_desc"
    #'similarNo' : None, # goodsNo와 동일
    # 'selectedSimilarNo' : None # 색상 아닌 전체
    }

musinsa_cmt_list = list()
# review_no_list = list()
review_no_dict = dict()
review_date_list = list()
review_date_dict = dict() # 키값이 리뷰넘버, 값이 스트링
review_image_dict = dict() # 키값이 리뷰넘버, 값이 url 리스트
review_recomment_dict = dict()
goods_list = list()
review_image_num_dict = dict()

for index in range(1000, 2000):
    goodsNo = tuples[index][1]
    search_pages = tuples[index][3]//40
    print(goodsNo, search_pages)
    review_no_list=list() # 위치 여기가 맞음
    for i in range(search_pages): # 페이지 range을 수정해야한다. 
        params['page'] = i+1
        params['goodsNo'] = goodsNo
        # params['similarNo'] = goodsNo
        # params['selectedSimilarNo'] = 0
        res = requests.get(url_good, params = params, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

            # 페이지마다 모두 뽑기
        comments = soup.find_all(attrs={"class":"comments comments-input"})
        reviews = soup.find_all(attrs={"class":"review-contents"})
        review_dates = soup.find_all("p", attrs={"class":"review-profile__date"})
        review_images = soup.find_all('div', attrs={"class" : "review-content-photo"})
#         print("상품", goodsNo)

           
        for idx,review in enumerate(reviews):  # 리뷰 넘버를 뽑는다
            review_no = review["data-review-no"]
            review_no_list.append(review_no)
            goods_list.append(goodsNo)
            review_no_dict[goodsNo] = review_no_list 
#             print("리뷰넘버 :", review_no)
                  

            for idx_date, date in enumerate(review_dates): # 리뷰 데이트를 뽑는다  
                if idx != idx_date:
                    pass
                elif idx == idx_date:
                    if len(date.text.split('.')) ==3:
                        review_date = (date.text)
                    else:
                        review_date = ("며칠전으로 표시된 리뷰")
            review_date_list.append(review_date)
            review_date_dict[review_no] = review_date



            for idx_cmt, comment in enumerate(comments): # 각 리뷰에 달린 무신사측 답글을 고르기
#                 print(idx_cmt)
                if idx != idx_cmt:
                    pass
                elif idx == idx_cmt:
                    find_recomments = comment.find_all("div", attrs={"class":"comment_recomment comment_wrap"}, recursive=True)
                    for idx_rcmt, find_recomment in enumerate(find_recomments):
#                         print("답글번호", idx_rcmt, find_recomment.find("span", attrs={"class":"cmt_summary"}))
                        if find_recomment.find("img", attrs={"alt":"무신사 관리자 썸네일"}) is not None: # 무신사가 답글 단 경우 
                            recomment_sum = find_recomment.find("span", attrs={"class":"cmt_summary"})
                            recomment_date = find_recomment.find("span", attrs={"class":"recomment_date"})
                            recomment_detail = find_recomment.find("span", attrs={"class":"cmt_detail"})
#                             print(recomment_sum, recomment_date, recomment_detail)
#                             print(review_date_dict[comment["c_idx"]].split('.'), recomment_date.text[:10].split('.'))

                            if reduce(lambda x,y: x+y, review_date_dict[comment["c_idx"]].split('.'))\
                                        > reduce(lambda x,y: x+y, recomment_date.text[:10].split('.')): # 리뷰날짜가 답글날짜보다 최신이면 정상
                                pass
                            else:
                                if recomment_detail:
                                    verifying_text = recomment_detail.text
                                    p = re.compile("안녕하세요\D+무신사\D+")
                                    o = re.compile("지급해드렸습니다")
                                    if p.search(verifying_text) is not None:
                                        if o.search(verifying_text) is None: # 지급해드렸습니다 답글 걸러냄
#                                             print(review_date_dict[comment["c_idx"]], recomment_date.text[:10], recomment_detail.text)

                                            musinsa_cmt_list.append(recomment_detail.text)
                                            review_recomment_dict[review_no] = recomment_detail.text

#             print(len(review_recomment_dict.keys()) )


             # 리뷰 사진을 뽑는다
            for idx_img, review_image_source in enumerate(review_images): # 10개의 리뷰
                review_img_pres = review_image_source.find_all("li", attrs={'class':'review-content-photo__item'})
                review_image_url_list = list()
                if idx != idx_img:
                    pass
                elif idx == idx_img: # 리뷰와 리뷰이미지 순서 매치
                    for idx_img_img, review_img_pre in enumerate(review_img_pres): # 한 리뷰당 복수의 이미지
                        if review_img_pre.find('img')["src"].startswith("//"):
                            review_img_url = ("https:" + review_img_pre.find('img')["src"][:-5])
                        review_image_url_list.append(review_img_url)
        #                 print(review_no, review_image_url_list)
                    review_image_dict[review_no] = review_image_url_list

                    review_image_num_dict[review_no] = len(review_image_url_list)



# print(len(review_image_dict.keys()))

# print(len(review_date_dict.keys()))

# print(len(review_recomment_dict.keys()))

# print(review_image_dict)

# print(review_date_dict)

# print(review_recomment_dict.keys())

# print("상품", goodsNo)

1911514 2
1894600 2
1959696 2
1387530 1
1958337 2
1408184 1
1387957 1
1915632 1
1447691 1
1904867 1
1319216 1
1633322 2
1031255 1
2247583 1
1913569 1
2461377 1
1633324 1
2468840 1
2731288 1
1939726 1
1144989 196
1129745 107
404474 109
1322019 90
1190264 84
407067 51
2149254 61
1130844 37
1190268 48
850153 34
710323 29
1272823 43
407066 31
996234 23
996233 25
1265162 38
1190282 21
394608 16
1129742 16
876277 18
847182 15
394585 11
434861 10
714846 8
1629101 9
2053660 17
816524 8
2058136 13
708937 5
1608272 9
1190265 10
1322020 9
1558846 13
2086668 11
1275020 8
1575853 10
1349496 12
604387 7
2058139 10
1144991 6
996240 5
404482 2
1000696 6
1383817 7
876246 6
1145013 7
1629100 4
1190278 6
853185 5
1145009 5
375788 3
744326 2
1144990 5
851452 3
877880 4
1144994 4
1173551 6
1524073 4
1571143 5
854753 3
607925 4
876556 3
1129743 3
1144996 3
1383415 5
1349500 5
1145010 3
854746 3
1341163 6
1310289 4
2390729 6
1145012 3
996235 3
1575855 5
1005536 4
1130042 3
823465 2
938858 3
961845 2
1341162 

572126 3
775076 1
2472412 6
1077033 4
996186 2
1503344 4
2443386 4
1698345 4
1932038 2
1854623 3
2329809 4
996499 2
1862475 4
2448091 5
1507159 3
1376670 5
2329308 5
1117543 2
1846927 3
2474511 4
1426300 3
1901121 4
1561528 4
547943 2
550829 2
2386335 4
2082064 5
1305510 3
1386488 2
340497 2
1876652 3
1842380 3
2377283 3
2458431 4
1294977 3
538281 2
2404509 3
340495 2
1804095 5
1908212 4
1841760 2
538283 2
2034139 5
2329286 5
1841758 3
2461466 3
1424102 2
1779734 3
1828020 3
1486972 3
1936059 4
2376224 3
1854796 2
1435204 2
2472145 4
2479604 4
996519 1
1385222 4
2461187 2
996511 1
996501 1
1925129 2
1365825 2
1866029 4
2371238 3
2024019 3
2377272 2
2424628 3
1378696 3
1058944 2
1876648 3
1010595 2
2398651 3
1869729 3
2368940 4
971740 1
1975155 2
540423 2
544238 1
1870545 2
1417717 3
2386334 3
1994271 4
1117542 2
791894 1
1345516 1
2368942 3
1117512 2
2476163 2
1521257 3
2442413 2
1427094 1
1412129 2
996500 1
1829238 3
1847029 2
994686 1
1960206 3
2404585 2
2455956 3
1051404 2
1117536 2

In [165]:

print(len(review_image_dict.keys()))

print(len(review_date_dict.keys()))

print(len(review_recomment_dict.keys()))


82106
82197
14569


In [166]:
len(review_no_dict.keys())

1000

In [111]:
review_no_dict["1165926"]

KeyError: '1165926'

In [170]:
con.close()

In [171]:
con = sqlite3.connect("./musinsa_1101.db")
cur = con.cursor()

In [173]:
con = sqlite3.connect("./musinsa_1102.db")
cur = con.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS 리뷰테이블;
    CREATE TABLE 리뷰테이블1000_1999(
        IDX INTEGER PRIMARY KEY AUTOINCREMENT,    
        goodsId INTEGER,
        reviewId INTEGER
    );
    
    DROP TABLE IF EXISTS 리뷰이미지;
    CREATE TABLE 리뷰이미지1000_1999(
        reviewId INTEGER,
        reviewImageUrl,
        reviewImageNum INTEGER
    );

    DROP TABLE IF EXISTS 리뷰날짜;
    CREATE TABLE 리뷰날짜1000_1999(
        reviewId INTEGER,
        reviewDate TEXT        
    );
    
    DROP TABLE IF EXISTS 리뷰답글;
    CREATE TABLE 리뷰답글1000_1999(
        reviewId INTEGER,
        reviewRecomment TEXT
    );
''')
con.commit()


In [80]:
review_no_dict

{'1781604': ['18470290',
  '25641670',
  '24153685',
  '34341207',
  '33429912',
  '33411745',
  '32679750',
  '29711578',
  '29477258',
  '29112603'],
 '1934636': ['27916369',
  '33784315',
  '32037941',
  '31998331',
  '31756915',
  '31225036',
  '29909375',
  '29817317',
  '29685602',
  '29485966'],
 '2149537': ['28589631',
  '34118178',
  '34010528',
  '33792452',
  '33516487',
  '33515837',
  '33479357',
  '33443056',
  '33426813',
  '33416779'],
 '2117248': ['34317316',
  '34214309',
  '34199084',
  '34140803',
  '34010678',
  '33825246',
  '33819872',
  '33573477',
  '33556739',
  '33450253'],
 '1248262': ['8009757',
  '7743423',
  '8990541',
  '15379658',
  '10734840',
  '33560968',
  '33049901',
  '32986055',
  '31893055',
  '31591810'],
 '1865808': ['22365131',
  '20812342',
  '34189155',
  '33982832',
  '33466453',
  '33243963',
  '32957476',
  '31827118',
  '31631809',
  '28324139'],
 '1811496': ['16070329',
  '16219217',
  '20125891',
  '17135772',
  '15918069',
  '1555644

In [174]:
db_review_no_list = [(list(review_no_dict.keys())[v] ,list(review_no_dict.values())[v][k])\
                     for v in range(len(list(review_no_dict.keys())))\
                      for k in range(len(list(review_no_dict.values())[v]))]


In [175]:
cur.executemany('''
    INSERT INTO 리뷰테이블1000_1999(goodsId, reviewId)
    VALUES(?, ?)
''', db_review_no_list)
con.commit()

In [176]:
db_review_image_list = [(list(review_image_dict.keys())[v] ,list(review_image_dict.values())[v][k])\
                     for v in range(len(list(review_image_dict.keys())))\
                      for k in range(len(list(review_image_dict.values())[v]))]

In [177]:
cur.executemany('''
    INSERT INTO 리뷰이미지1000_1999(reviewId, reviewImageUrl)
    VALUES(?, ?)
''', db_review_image_list)
con.commit()

In [44]:
review_date_dict

{'17538638': '2021.06.16',
 '33957126': '며칠전으로 표시된 리뷰',
 '33749076': '며칠전으로 표시된 리뷰',
 '33627968': '며칠전으로 표시된 리뷰',
 '27465337': '2022.04.20',
 '26571707': '2022.03.24',
 '26547743': '2022.03.23',
 '26420363': '2022.03.19',
 '26294625': '2022.03.16',
 '26013450': '2022.03.08',
 '25541344': '2022.02.22',
 '34117432': '며칠전으로 표시된 리뷰',
 '34115566': '며칠전으로 표시된 리뷰',
 '33975559': '며칠전으로 표시된 리뷰',
 '33281788': '며칠전으로 표시된 리뷰',
 '32607228': '2022.09.20',
 '32552711': '2022.09.18',
 '30474554': '2022.07.18',
 '29666773': '2022.06.23',
 '29041128': '2022.06.04',
 '27772379': '2022.04.29',
 '27415795': '2022.04.19',
 '27382453': '2022.04.18',
 '27194392': '2022.04.12',
 '27078828': '2022.04.09',
 '27051515': '2022.04.08',
 '26980642': '2022.04.06',
 '26932084': '2022.04.04',
 '26833385': '2022.04.01',
 '26809216': '2022.03.31',
 '24887853': '2022.02.04',
 '23587693': '2021.12.21',
 '19323355': '2021.08.28',
 '34066286': '며칠전으로 표시된 리뷰',
 '33652057': '며칠전으로 표시된 리뷰',
 '33393013': '며칠전으로 표시된 리뷰',
 '330384

In [98]:
review_date_dict[list(review_date_dict.keys())[1]]

'2019.02.27'

In [178]:
db_review_date_list = [(list(review_date_dict.keys())[v] ,review_date_dict[list(review_date_dict.keys())[v]])\
                     for v in range(len(list(review_date_dict.keys())))]

In [46]:
db_review_date_list

[('17538638', '2021.06.16'),
 ('33957126', '며칠전으로 표시된 리뷰'),
 ('33749076', '며칠전으로 표시된 리뷰'),
 ('33627968', '며칠전으로 표시된 리뷰'),
 ('27465337', '2022.04.20'),
 ('26571707', '2022.03.24'),
 ('26547743', '2022.03.23'),
 ('26420363', '2022.03.19'),
 ('26294625', '2022.03.16'),
 ('26013450', '2022.03.08'),
 ('25541344', '2022.02.22'),
 ('34117432', '며칠전으로 표시된 리뷰'),
 ('34115566', '며칠전으로 표시된 리뷰'),
 ('33975559', '며칠전으로 표시된 리뷰'),
 ('33281788', '며칠전으로 표시된 리뷰'),
 ('32607228', '2022.09.20'),
 ('32552711', '2022.09.18'),
 ('30474554', '2022.07.18'),
 ('29666773', '2022.06.23'),
 ('29041128', '2022.06.04'),
 ('27772379', '2022.04.29'),
 ('27415795', '2022.04.19'),
 ('27382453', '2022.04.18'),
 ('27194392', '2022.04.12'),
 ('27078828', '2022.04.09'),
 ('27051515', '2022.04.08'),
 ('26980642', '2022.04.06'),
 ('26932084', '2022.04.04'),
 ('26833385', '2022.04.01'),
 ('26809216', '2022.03.31'),
 ('24887853', '2022.02.04'),
 ('23587693', '2021.12.21'),
 ('19323355', '2021.08.28'),
 ('34066286', '며칠전으로 표시된 리뷰')

In [27]:
len(db_review_date_list)

38171

In [179]:
cur.executemany('''
    INSERT INTO 리뷰날짜1000_1999(reviewId, reviewDate)
    VALUES(?, ?)
''', db_review_date_list)
con.commit()

In [110]:
len(review_recomment_dict.keys())

210

In [111]:
len(review_recomment_dict.values())

210

In [180]:
db_review_recomment_list = [(list(review_recomment_dict.keys())[v] ,review_recomment_dict[list(review_recomment_dict.keys())[v]])\
                     for v in range(len(list(review_recomment_dict.keys())))]

In [49]:
db_review_recomment_list

[('17538638',
  '안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다. 지급 보류 유형 7. 전체 실루엣 식별 어려운 컷▶ 지급 보류 상세 사유 : 접힌 부분을 펼쳐서 상품의 형태가 프레임 내에 또렷하게 보이도록 직접 촬영한 사진으로 수정 부탁드립니다. (소매, 끝부분 등 부분적으로 잘리지 않아야 함)TIP. 상품 사진 후기는 상품을 정밀하게 확인하실 수 있도록 만든 후기로 사진 등록 시 상품 전체 실루엣이 프레임 안에 전부 보여야 하며,신체의 일부만 등록되거나 상품 식별이 불가능한 경우 지급이 보류될 수 있습니다.특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 상품 사진 후기 적립금 지급 조건 -  겉 포장을 제거하여 접힌 부분을 펼쳐서 상품의 형태가 프레임 내에 또렷하게 보이도록 직접 촬영한 사진-  20자 이상의 상품에 대한 후기  ▶ 상품평 수정 방법 : PC, APP :  마이페이지 → 후기 작성 → 후기 내역(수정 희망하시는 후기 내용 클릭) → 수정/삭제※ 후기 적립금은 자동 지급이 아닌, 수기로 지급되며 영업일 기준 1~2일 정도 소요되는 점 양해 부탁드립니다.※ 댓글은 관리자가 확인하지 않습니다. 후기 관련하여 궁금하신 내용이 있으실 경우에는 1:1문의로 문의 부탁드립니다.※ 해당 후기는 정상적으로 노출됩니다.'),
 ('25541344',
  '안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다. 지급 보류 유형 4.다른 상품 컷▶ 지급 보류 상세 사유 : 루즈 핏 스웨트 

In [181]:
cur.executemany('''
    INSERT INTO 리뷰답글1000_1999(reviewId, reviewRecomment)
    VALUES(?, ?)
''', db_review_recomment_list)
con.commit()

In [182]:
cur.execute('''
    SELECT D.goodsId, D.reviewId, B.reviewDate, A.reviewImageUrl, C.reviewRecomment
    FROM 리뷰테이블1000_1999 AS D
    LEFT JOIN 리뷰날짜1000_1999 AS B
    ON D.reviewId = B.reviewId
    LEFT JOIN 리뷰답글1000_1999 AS C
    ON D.reviewId = C.reviewId
    LEFT JOIN 리뷰이미지1000_1999 AS A
    ON D.reviewId = A.reviewId;
    

''').fetchall()

[(1911514,
  31072367,
  '2022.08.04',
  'https://image.msscdn.net/data/estimate/1911514_0/gallery_62eb908dd40a3.jpg',
  '안녕하세요. 패션을 사랑하는 무신사 CS입니다.정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다. 지급 보류 유형 8.전신 실루엣 식별 어려운 컷▶ 지급 보류 상세 사유 : 위에서 찍은 사진으로 전신 확인이 정확하게 어려운 사진 컷으로 확인됩니다. 하여, 정면의 전신 컷을 추가 부탁드립니다.Tip. 얼굴은 제외 가능하며, 어깨부터 발끝까지 프레임 안에 담겨야 합니다.착용이 어려운 상품은 휴대 상태 전신 컷 또는 전신 스타일링 컷 1장과 구매하신 상품 단독 컷 1장으로 올려주시면 적립금 지급이 가능합니다.(예:이너웨어, 화장품, 스마트기기, 옷걸이 등등)특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 스타일 후기 적립금 지급 조건 - 반드시 구매하신 상품을 착용한 전신 스타일링 컷- 착용이 어려운 상품은 전신 스타일링한 상태에서 해당 상품이 보이도록 든 상태의 전신 컷 - 20자 이상의 상품에 대한 후기 ▶ 상품평 수정 방법- PC, APP :  마이페이지 → 후기 작성 → 후기 내역(수정 희망하시는 후기 내용 클릭) → 수정/삭제※ 후기 적립금은 자동 지급이 아닌, 수기로 지급되며 영업일 기준 1~2일 정도 소요되는 점 양해 부탁드립니다.※ 댓글은 관리자가 확인하지 않습니다. 후기 관련하여 궁금하신 내용이 있으실 경우에는 1:1문의로 문의 부탁드립니다.※ 해당 후기는 정상적으로 노출됩니다.문의하신 사항 외에 서비스 이용 관련 내용은 FAQ를 통해 상세하게 확인하실 수 있습니다.FAQ ▶ https://www.musinsa.com/app/c

In [184]:
cur.execute('''
    create view 스타일리뷰1000_1999 as 
    SELECT D.goodsId, D.reviewId, B.reviewDate, A.reviewImageUrl, C.reviewRecomment
    FROM 리뷰테이블1000_1999 AS D
    LEFT JOIN 리뷰날짜1000_1999 AS B
    ON D.reviewId = B.reviewId
    LEFT JOIN 리뷰답글1000_1999 AS C
    ON D.reviewId = C.reviewId
    LEFT JOIN 리뷰이미지1000_1999 AS A
    ON D.reviewId = A.reviewId;
''')

In [185]:
con.commit()

In [164]:
con = sqlite3.connect("./musinsa_1101.db")
cur = con.cursor()
sql = cur.execute('''
    SELECT goodsId,
    COUNT(goodsId)
    FROM 리뷰테이블1_1000
    GROUP BY goodsId
''').fetchall()
            

In [156]:
len(sql)

1000

In [205]:
con.close()

In [206]:
con = sqlite3.connect("./musinsa_1101.db")
cur = con.cursor()

In [207]:
cur.execute('''
    ATTACH DATABASE "./musinsa_1102.db" AS connectionTest;
''')
con.commit()

In [212]:
con.close()

In [213]:
con = sqlite3.connect("./musinsa_style_review.db")
cur = con.cursor()
cur.executescript('''
    CREATE TABLE 리뷰테이블1000_1999(
        IDX INTEGER PRIMARY KEY AUTOINCREMENT,    
        goodsId INTEGER,
        reviewId INTEGER
    );
    
    CREATE TABLE 리뷰이미지1000_1999(
        reviewId INTEGER,
        reviewImageUrl,
        reviewImageNum INTEGER
    );

    CREATE TABLE 리뷰날짜1000_1999(
        reviewId INTEGER,
        reviewDate TEXT        
    );
    
    CREATE TABLE 리뷰답글1000_1999(
        reviewId INTEGER,
        reviewRecomment TEXT
    );
''')
con.commit()


In [214]:
cur.execute('''
    ATTACH DATABASE "./musinsa_1102.db" AS connectionTest;
''')
con.commit()

In [215]:
cur.execute('''
    INSERT INTO 리뷰테이블1000_1999 SELECT * FROM connectionTest.리뷰테이블1000_1999;
''')

In [216]:
con.commit()

In [218]:
cur.executescript('''
    INSERT INTO 리뷰이미지1000_1999 SELECT * FROM connectionTest.리뷰이미지1000_1999;
    INSERT INTO 리뷰답글1000_1999 SELECT * FROM connectionTest.리뷰답글1000_1999;
    INSERT INTO 리뷰날짜1000_1999 SELECT * FROM connectionTest.리뷰날짜1000_1999;
''')
con.commit()

In [219]:
cur.execute('''
    ATTACH DATABASE "./musinsa_1101.db" AS connectionDB1;
''')
con.commit()

In [232]:
cur.executescript('''
    CREATE TABLE 리뷰테이블5000_5191(
        IDX INTEGER PRIMARY KEY AUTOINCREMENT,    
        goodsId INTEGER,
        reviewId INTEGER
    );
    
    CREATE TABLE 리뷰이미지5000_5191(
        reviewId INTEGER,
        reviewImageUrl,
        reviewImageNum INTEGER
    );

    CREATE TABLE 리뷰날짜5000_5191(
        reviewId INTEGER,
        reviewDate TEXT        
    );
    
    CREATE TABLE 리뷰답글5000_5191(
        reviewId INTEGER,
        reviewRecomment TEXT
    );
''')
con.commit()

In [233]:
cur.executescript('''
    INSERT INTO 리뷰이미지5000_5191 SELECT * FROM connectionDB1.리뷰이미지5000_5191;
    INSERT INTO 리뷰답글5000_5191 SELECT * FROM connectionDB1.리뷰답글5000_5191;
    INSERT INTO 리뷰날짜5000_5191 SELECT * FROM connectionDB1.리뷰날짜5000_5191;
    INSERT INTO 리뷰테이블5000_5191 SELECT * FROM connectionDB1.리뷰테이블5000_5191;
''')
con.commit()

In [222]:
cur.execute('''
    INSERT INTO 리뷰테이블1_1000 SELECT * FROM connectionDB1.리뷰테이블1_1000;
''')
con.commit()

In [236]:
cur.execute('''
    CREATE TABLE 리뷰테이블_합침테스트_1 AS
    SELECT * FROM 리뷰테이블1_1000
    UNION ALL
    SELECT * FROM 리뷰테이블1000_1999
    UNION ALL
    SELECT * FROM 리뷰테이블2000_2999;
''')
con.commit()

In [237]:
cur.executescript('''
    DROP TABLE IF EXISTS 리뷰테이블_합침테스트_1;
    DROP TABLE IF EXISTS 리뷰테이블_합침테스트;
''')
con.commit()

In [239]:
cur.executescript('''
    CREATE TABLE 리뷰테이블_전체 AS
    SELECT * FROM 리뷰테이블1_1000
    UNION ALL
    SELECT * FROM 리뷰테이블1000_1999
    UNION ALL
    SELECT * FROM 리뷰테이블2000_2999
    UNION ALL
    SELECT * FROM 리뷰테이블3000_3999
    UNION ALL
    SELECT * FROM 리뷰테이블4000_4499
    UNION ALL
    SELECT * FROM 리뷰테이블4500_4999
    UNION ALL
    SELECT * FROM 리뷰테이블5000_5191;
    
    CREATE TABLE 리뷰답글_전체 AS
    SELECT * FROM 리뷰답글1_1000
    UNION ALL
    SELECT * FROM 리뷰답글1000_1999
    UNION ALL
    SELECT * FROM 리뷰답글2000_2999
    UNION ALL
    SELECT * FROM 리뷰답글3000_3999
    UNION ALL
    SELECT * FROM 리뷰답글4000_4499
    UNION ALL
    SELECT * FROM 리뷰답글4500_4999
    UNION ALL
    SELECT * FROM 리뷰답글5000_5191;
    
    CREATE TABLE 리뷰이미지_전체 AS
    SELECT * FROM 리뷰이미지1_1000
    UNION ALL
    SELECT * FROM 리뷰이미지1000_1999
    UNION ALL
    SELECT * FROM 리뷰이미지2000_2999
    UNION ALL
    SELECT * FROM 리뷰이미지3000_3999
    UNION ALL
    SELECT * FROM 리뷰이미지4000_4499
    UNION ALL
    SELECT * FROM 리뷰이미지4500_4999
    UNION ALL
    SELECT * FROM 리뷰이미지5000_5191;
    
    CREATE TABLE 리뷰날짜_전체 AS
    SELECT * FROM 리뷰날짜1_1000
    UNION ALL
    SELECT * FROM 리뷰날짜1000_1999
    UNION ALL
    SELECT * FROM 리뷰날짜2000_2999
    UNION ALL
    SELECT * FROM 리뷰날짜3000_3999
    UNION ALL
    SELECT * FROM 리뷰날짜4000_4499
    UNION ALL
    SELECT * FROM 리뷰날짜4500_4999
    UNION ALL
    SELECT * FROM 리뷰날짜5000_5191;
''')
con.commit()

In [240]:
cur.execute('''
    CREATE VIEW 스타일리뷰_전체 as 
    SELECT D.goodsId, D.reviewId, B.reviewDate, A.reviewImageUrl, C.reviewRecomment
    FROM 리뷰테이블_전체 AS D
    LEFT JOIN 리뷰날짜_전체 AS B
    ON D.reviewId = B.reviewId
    LEFT JOIN 리뷰답글_전체 AS C
    ON D.reviewId = C.reviewId
    LEFT JOIN 리뷰이미지_전체 AS A
    ON D.reviewId = A.reviewId;
''')
con.commit()

In [243]:
from pandas_profiling import ProfileReport

In [244]:
report = ProfileReport()

In [ ]:
report

In [245]:
ProfileReport(data)

NameError: name 'data' is not defined

In [246]:
con.close()

In [249]:
con = sqlite3.connect("./musinsa_style_reivew.db")
cur = con.cursor()

In [4]:
cur.execute('''
    ATTACH DATABASE "musinsa_1101.db" AS connectionCate;
''')
con.commit()

In [252]:
cur.execute('''
    DROP TABLE IF EXISTS 소분류테이블
    CREATE TABLE 소분류테이블(
    idx INTEGER,
    major_category TEXT,
    mid_category TEXT,
    goods_id TEXT,
    goods_url TEXT,
    goods_image TEXT
);
''')

In [253]:
cur.execute('''
    INSERT INTO 소분류테이블 SELECT * FROM connectionCate.소분류테이블;
''')
con.commit()

In [12]:
cur.execute('''
    CREATE VIEW 스타일리뷰_EDA as 
    SELECT * FROM 스타일리뷰_전체_레이블 AS A
    LEFT JOIN 소분류테이블 AS B
    ON A.goodsId = B.goods_id;
''')
con.commit()

In [14]:
import pandas as pd
from pandas_profiling import ProfileReport

In [16]:
data = pd.read_csv("style_reivew_EDA.csv")

In [17]:
data

goodsId  reviewId  reviewDate  \
0        947067   7997463  2020.01.21   
1        947067   4067306  2019.02.27   
2        947067   8468969  2020.02.28   
3        947067   7182239  2019.11.20   
4        947067   7001487  2019.11.08   
...         ...       ...         ...   
399552  1165926  30180427  2022.07.08   
399553  1165926  30180427  2022.07.08   
399554  1165926  30145259  2022.07.07   
399555  1165926  29187788  2022.06.09   
399556  1165926  28858969  2022.05.30   

                                           reviewImageUrl  \
0       https://image.msscdn.net/data/estimate/947067_...   
1       https://image.msscdn.net/data/estimate/947067_...   
2       https://image.msscdn.net/data/estimate/947067_...   
3       https://image.msscdn.net/data/estimate/947067_...   
4       https://image.msscdn.net/data/estimate/947067_...   
...                                                   ...   
399552  https://image.msscdn.net/data/estimate/1165926...   
399553  https://image.msscdn.net/data/estimate/1165926...   
399554  https://image.msscdn.net/data/estimate/1165926...   
399555  https://image.msscdn.net/data/estimate/1165926...   
399556  https://image.msscdn.net/data/estimate/1165926...   

                                          reviewRecomment  label    idx  \
0                                                     NaN    NaN      0   
1                                                     NaN    NaN      0   
2                                                     NaN    NaN      0   
3       안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기...    4.0      0   
4       안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기...    4.0      0   
...                                                   ...    ...    ...   
399552                                                NaN    NaN  34153   
399553                                                NaN    NaN  34153   
399554                                                NaN    NaN  34153   
399555                                                NaN    NaN  34153   
399556                                                NaN    NaN  34153   

        major_category  mid_category  goods_id  \
0                    1          1005    947067   
1                    1          1005    947067   
2                    1          1005    947067   
3                    1          1005    947067   
4                    1          1005    947067   
...                ...           ...       ...   
399552               3          3005   1165926   
399553               3          3005   1165926   
399554               3          3005   1165926   
399555               3          3005   1165926   
399556               3          3005   1165926   

                                        goods_url  \
0        https://www.musinsa.com/app/goods/947067   
1        https://www.musinsa.com/app/goods/947067   
2        https://www.musinsa.com/app/goods/947067   
3        https://www.musinsa.com/app/goods/947067   
4        https://www.musinsa.com/app/goods/947067   
...                                           ...   
399552  https://www.musinsa.com/app/goods/1165926   
399553  https://www.musinsa.com/app/goods/1165926   
399554  https://www.musinsa.com/app/goods/1165926   
399555  https://www.musinsa.com/app/goods/1165926   
399556  https://www.musinsa.com/app/goods/1165926   

                                              goods_image  
0       https://image.msscdn.net/images/goods_img/2019...  
1       https://image.msscdn.net/images/goods_img/2019...  
2       https://image.msscdn.net/images/goods_img/2019...  
3       https://image.msscdn.net/images/goods_img/2019...  
4       https://image.msscdn.net/images/goods_img/2019...  
...                                                   ...  
399552  https://image.msscdn.net/images/goods_img/2019...  
399553  https://image.msscdn.net/images/goods_img/2019...  
399554  https://image.msscdn.net/images/goods_img/2019...  
399555  https://image.msscdn.net/images/goo

In [19]:
PR = ProfileReport(data)

In [20]:
PR.to_file('./PR.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\panda\anaconda3\lib\site-packages\scipy\stats\stats.py:4812: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [141]:
musinsa_cmt_list

['안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ 스타일 후기 기준스타일 후기의 취지는 단순히 상품만을 공유하는 것이 아닌, 고객님의 패션 스타일을 다른 고객분들과 함께 공유하고자 하는 것입니다.① 구매하신 상품을 착용한 전신 스타일링 컷(얼굴 제외 가능, 어깨부터 발끝까지 프레임 안에 전부 보이도록 촬영한 사진)② 착용이 어려운 상품은 전신 스타일링한 상태에서 제품이 보이도록 든 상태의 사진③ 20자 이상의 상품에 대한 후기 수고로우시겠지만, 위 기준을 참고하여 수정해주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 메뉴(≡) → 구매 후기 → 후기 내역 → 수정 희망 하시는 후기 내용 클릭 → 수정 ② PC : 무신사 홈페이지 → 로그인 → 마이페이지 → 구매 후기 → 후기 내역 → 수정 감사합니다.',
 '안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ 스타일 후기 기준스타일 후기의 취지는 단순히 상품만을 공유하는 것이 아닌, 고객님의 패션 스타일을 다른 고객분들과 함께 공유하고자 하는 것입니다.① 구매하신 상품을 착용한 전신 스타일링 컷(얼굴 제외 가능, 어깨부터 발끝까지 프레임 안에 전부 보이도록 촬영한 사진)② 착용이 어려운 상품은 전신 스타일링한 상태에서 제품이 보이도록 든 상태의 사진③ 20자 이상의 상품에 대한 후기 수고로우시겠지만, 위 기준을 참고하여 수정해주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 메뉴(≡) → 구매 후기 → 후기 내역 → 수정 희망 하시는 후기 내용 클릭 → 수정 ② PC : 무신사 홈페이지 → 로그인 → 마이페이지 → 구매 후기 → 후기 내역 → 수정 감사합니다.',
 '

In [142]:
len(musinsa_cmt_list), len(set(musinsa_cmt_list))

(211, 53)

In [160]:
for i in set(musinsa_cmt_list):
    p = re.compile("▶.+?지급.+?보류(.+)")
    q = re.compile("지급.+?보류")
    group1 = re.split(q,i)
#     group = re.search(p,i).group(1)
    if group1:
        
        print(group1)

['안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 ', '된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 ', ' 유형 7.전체 실루엣 식별 어려운 컷▶ ', ' 상세사유 : 구매하신 상품의 식별이 어려운 사진 컷 사용TIP. 얼굴은 제외 가능하며, 어깨부터 발끝까지 프레임 안에 담겨야 합니다.착용이 어려운 상품은 휴대 상태 전신 컷 또는 전신 스타일링 컷 1장과 구매하신 상품 단독 컷 1장으로 올려주시면 적립금 지급이 가능합니다.(예:이너웨어, 화장품, 스마트기기, 옷걸이 등등)특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 스타일 후기 적립금 지급 조건 - 반드시 구매하신 상품을 착용한 전신 스타일링 컷- 착용이 어려운 상품은 전신 스타일링한 상태에서 해당 상품이 보이도록 든 상태의 전신 컷 - 20자 이상의 상품에 대한 후기 ▶ 상품평 수정 방법- PC, APP :  마이페이지 → 후기 작성 → 후기 내역(수정 희망하시는 후기 내용 클릭) → 수정/삭제※ 후기 적립금은 자동 지급이 아닌, 수기로 지급되며 영업일 기준 1~2일 정도 소요되는 점 양해 부탁드립니다.※ 댓글은 관리자가 확인하지 않습니다. 후기 관련하여 궁금하신 내용이 있으실 경우에는 1:1문의로 문의 부탁드립니다.※ 해당 후기는 정상적으로 노출됩니다.']
['안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 ', '상태입니다.▶ 스타일 후기 기준스타일 후기의 취지는 단순히 상품만을 공유하는 것이 아닌, 고객님의 패션 스타일을 다른 고객분들과 함께 공유하고자 하는 것입니다.① 구매하신 상품을 착용한 전신 스타일링 컷-얼굴 제외 가능, ★어깨부터 발끝까지★ 프레임 안에 전부 보이도록 촬영한 사진 (모자이크,소품,거울 등 끝부분이 잘리지 않도록 촬영)② 착용이 어려운 상품은 전신 스타일링한 상태에서 

In [5]:
import sqlite3

In [6]:
con = sqlite3.connect("musinsa_style_review.db")
cur = con.cursor()

In [9]:
cur.executescript('''
    DROP TABLE IF EXISTS 소분류테이블;
    CREATE TABLE 소분류테이블(
    idx INTEGER,
    major_category TEXT,
    mid_category TEXT,
    goods_id INTEGER,
    goods_url TEXT,
    goods_image TEXT
);
''')
con.commit()

In [10]:
cur.execute('''
    ATTACH DATABASE "musinsa_1101.db" AS connectionCate;
''')
con.commit()

In [11]:
cur.execute('''
    INSERT INTO 소분류테이블 SELECT * FROM connectionCate.소분류테이블;
''')
con.commit()